In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
import cv2
import os 
import keras
from keras import layers, models
from keras import save_model
from tensorflow import keras as tfkeras
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm 

In [ ]:
image_size = 128
batch_size = 64
learn_rate = 0.01
epochs = 100
data_dir = "/kaggle/input/plantvillage-dataset/color"
input_folder = "/kaggle/input/plant-disease-test/test_images"
test_images = os.listdir("/kaggle/input/plant-disease-test/test_images")

In [ ]:
def read_img(image_data):
    try:
        image = cv2.imread(image_data)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (image_size, image_size))
        image = np.array(image, dtype=np.float32) / 255.0  # Normalize the image
        image = image.reshape(1, image_size, image_size, 3)
        
        return image

    except Exception as e:
        print(f"Error> {e}")

In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.1,
    subset="training",
    seed=123,
    image_size=(image_size, image_size),
    batch_size=batch_size,
)

val_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.1,
    subset="validation",
    seed=123,
    image_size=(image_size, image_size),
    batch_size=batch_size,
)

In [ ]:
lencoder = LabelEncoder()

disease_classes = train_data.class_names

enc_classes = lencoder.fit_transform(disease_classes)

encoded_disease_classes = lencoder.classes_

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_data.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_data.cache().prefetch(buffer_size=AUTOTUNE)

optimizer = keras.optimizers.Adam(learning_rate=learn_rate)

early_stop = tfkeras.callbacks.EarlyStopping(
    min_delta=0.001, patience=10, restore_best_weights=True
)

In [ ]:
disease_clf = models.Sequential(
    [
        layers.Rescaling(1.0 / 255, input_shape=(image_size, image_size, 3)),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dense(38, activation="softmax"),
    ]
)

disease_clf.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

disease_clf.summary()

In [ ]:
%%time

history = disease_clf.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
)

In [ ]:
test_accuracy = disease_clf.evaluate(val_data, batch_size=32)

print(
    f"The trained plant leaf disease detection model has an accuracy of {100*test_accuracy[1]}%"
)

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs_range = range(epochs)

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend(loc="upper right")
plt.title("Training and Validation Accuracy")

In [ ]:
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
disease_clf.save("tf_plant_disease_classifier.keras")
disease_clf.save("tf_plant_disease_classifier.h5")

In [ ]:
def process_image(imgpath):
    fileImage = Image.open(imgpath).convert("RGB").resize([width, height], Image.LANCZOS)  # type: ignore
    image = np.array(fileImage)
    img_array = image.reshape(1, width, height, 3)
    img_array = img_array.astype("float32")
    img_array = img_array / 255.0

    return img_array


image_classifier = loadmodel()
prediction = predict(img_path)
predicted_class = prediction.argmax()
certainty = 100 * prediction.max()

In [ ]:
image_list = [os.path.join(input_folder, image_file) for image_file in test_images]

def detect_plant_disease(image: str):
    image = read_img(image)
    class_score = disease_clf.predict(image)
    
    predicted_class = class_score.argmax()
    certainty = 100 * class_score.max()
    
    return predicted_class, certainty
    
    
for image in tqdm(image_list):
    disease_class, certainty = detect_plant_disease(image)
    
    print(f'Input Sentence: {image}')
    print(f'Predicted: {encoded_disease_classes[disease_class]}')
    print(f'Certainty: {certainty:.2f}%')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(disease_clf)
tflite_model = converter.convert()


with open("leaf_disease_clf.tflite", "wb") as f:
    f.write(tflite_model)